# Laboratorio: Aprendizaje Supervisado

### Integrantes:
### Javier Benitez 23405, Javier Lopéz 


pip install scikit-learn

In [31]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

housing_data = fetch_california_housing(as_frame=True)

df = housing_data.frame
X = housing_data.data
y = housing_data.target

In [32]:
print(df.describe())

             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486      5.429000      1.096675   1425.476744   
std        1.899822     12.585558      2.474173      0.473911   1132.462122   
min        0.499900      1.000000      0.846154      0.333333      3.000000   
25%        2.563400     18.000000      4.440716      1.006079    787.000000   
50%        3.534800     29.000000      5.229129      1.048780   1166.000000   
75%        4.743250     37.000000      6.052381      1.099526   1725.000000   
max       15.000100     52.000000    141.909091     34.066667  35682.000000   

           AveOccup      Latitude     Longitude   MedHouseVal  
count  20640.000000  20640.000000  20640.000000  20640.000000  
mean       3.070655     35.631861   -119.569704      2.068558  
std       10.386050      2.135952      2.003532      1.153956  
min        0.692308     32.54000

## Breve Analisis

La variable objetivo es MedHouseVal y se puede ver que los datos estan bastante agrupados a que tienen un std cercano a 1


In [33]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

print(f"Total de datos: {len(X)}")
print(f"Entrenamiento: {len(X_train)}")
print(f"Prueba: {len(X_test)}")

Total de datos: 20640
Entrenamiento: 16512
Prueba: 4128


## Justificar los valores usados para test_size y random_state.

Se uso el 20% ya que es el valor "optimo" de datos segun la regla de pareto que el 80% de los resultados vienen de el primer 20% y si usamos mas datos como el 50% se estarian desperdiciando datos valiosos que el modelo nececita para aprender.

Y para el random state ya que actua como una "semilla" para la reproducibilidad de los datos.

In [34]:
#usando regresion linear
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## Utilizando MAE

In [35]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

MAE: 0.5313679167350401


## Utilizando MSE

In [36]:
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

MSE: 0.5278524621633068


## Utilizando RMSE

In [37]:
rmse = mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")

RMSE: 0.5278524621633068


## R^2

In [38]:
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: 0.6075494500520068


## Optimizacion de Hiperparámetros
Usando GridSearchCV

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

modelo = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(
    estimator=modelo, 
    param_grid=param_grid, 
    cv=5,              
    scoring='r2',      
    n_jobs=6         
)

grid_search.fit(X_train, y_train)

print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación (R2): {grid_search.best_score_}")

Mejores parámetros: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}
Mejor puntuación (R2): 0.8053803164663019


## Justificar la elección del método de búsqueda y del esquema de validación cruzada

Se optó por GridSearchCV debido a que el espacio de búsqueda de hiperparámetros es manejable y se busca garantizar el hallazgo de la combinación óptima de forma exhaustiva.

para asegurar que los parámetros seleccionados no dependan de una partición al azar de los datos se opto por el esquema validación cruzada de 5 pliegues

# Problema de Clasificación Binaria
##### Breast Cancer Wisconsin Dataset

In [40]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Dataset
from sklearn.datasets import load_breast_cancer

# Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Evaluación
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

# Optimización
from sklearn.model_selection import GridSearchCV

### Dataset

In [41]:
data = load_breast_cancer(as_frame=True)
df = data.frame

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


### Exploración de los datos

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [43]:
df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


#### Balanceo de clases

In [44]:
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In [45]:
df['target'].value_counts(normalize=True)

target
1    0.627417
0    0.372583
Name: proportion, dtype: float64

### Separación de datos

In [46]:
X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


#### Escalamiento

In [47]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Entrenamiento

In [48]:
log_reg = LogisticRegression(max_iter=10000)
log_reg.fit(X_train_scaled, y_train)

y_pred_lr = log_reg.predict(X_test_scaled)

In [49]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled)

In [50]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

### Evaluacion

In [51]:
def evaluate_model(y_test, y_pred, model_name):
    print(f"--- {model_name} ---")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-score:", f1_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\n")


In [52]:
evaluate_model(y_test, y_pred_lr, "Logistic Regression")
evaluate_model(y_test, y_pred_knn, "kNN")
evaluate_model(y_test, y_pred_rf, "Random Forest")

--- Logistic Regression ---
Accuracy: 0.9824561403508771
Precision: 0.9861111111111112
Recall: 0.9861111111111112
F1-score: 0.9861111111111112
Confusion Matrix:
 [[41  1]
 [ 1 71]]


--- kNN ---
Accuracy: 0.956140350877193
Precision: 0.958904109589041
Recall: 0.9722222222222222
F1-score: 0.9655172413793104
Confusion Matrix:
 [[39  3]
 [ 2 70]]


--- Random Forest ---
Accuracy: 0.956140350877193
Precision: 0.958904109589041
Recall: 0.9722222222222222
F1-score: 0.9655172413793104
Confusion Matrix:
 [[39  3]
 [ 2 70]]




## Hiperparametros

In [53]:
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear']
}

grid_lr = GridSearchCV(
    LogisticRegression(max_iter=10000),
    param_grid_lr,
    cv=5,
    scoring='f1'
)

grid_lr.fit(X_train_scaled, y_train)

grid_lr.best_params_


{'C': 0.1, 'solver': 'lbfgs'}

In [54]:
best_lr = grid_lr.best_estimator_
y_pred_lr_opt = best_lr.predict(X_test_scaled)

evaluate_model(y_test, y_pred_lr_opt, "Logistic Regression Optimized")

--- Logistic Regression Optimized ---
Accuracy: 0.9736842105263158
Precision: 0.9726027397260274
Recall: 0.9861111111111112
F1-score: 0.9793103448275862
Confusion Matrix:
 [[40  2]
 [ 1 71]]




#### kNN

In [55]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid_knn,
    cv=5,
    scoring='f1'
)

grid_knn.fit(X_train_scaled, y_train)

grid_knn.best_params_

{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}

In [56]:
best_knn = grid_knn.best_estimator_
y_pred_knn_opt = best_knn.predict(X_test_scaled)

evaluate_model(y_test, y_pred_knn_opt, "kNN Optimized")

--- kNN Optimized ---
Accuracy: 0.9824561403508771
Precision: 0.972972972972973
Recall: 1.0
F1-score: 0.9863013698630136
Confusion Matrix:
 [[40  2]
 [ 0 72]]




#### Random Forest

In [57]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    cv=5,
    scoring='f1'
)

grid_rf.fit(X_train, y_train)

grid_rf.best_params_

{'max_depth': None,
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 100}

In [58]:
best_rf = grid_rf.best_estimator_
y_pred_rf_opt = best_rf.predict(X_test)

evaluate_model(y_test, y_pred_rf_opt, "Random Forest Optimized")

--- Random Forest Optimized ---
Accuracy: 0.956140350877193
Precision: 0.958904109589041
Recall: 0.9722222222222222
F1-score: 0.9655172413793104
Confusion Matrix:
 [[39  3]
 [ 2 70]]




## Analisis

In [59]:
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': best_lr.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

coef_df.head()

,Feature,Coefficient
19,fractal dimension error,0.229461
15,compactness error,0.227123
9,mean fractal dimension,0.214020
18,symmetry error,0.150838
16,concavity error,0.078932


In [60]:
importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

importances.head()


,Feature,Importance
23,worst area,0.147168
27,worst concave points,0.135973
20,worst radius,0.103521
7,mean concave points,0.101476
22,worst perimeter,0.075447


# Reflexión Final del Laboratorio

En este laboratorio trabajamos un problema real de clasificación binaria utilizando el dataset Breast Cancer Wisconsin. El objetivo fue entrenar distintos modelos de Machine Learning para predecir si un tumor es benigno o maligno, y analizar cuál de ellos funciona mejor considerando no solo el rendimiento numérico, sino también el impacto de los errores.

Primero exploramos el dataset para entender su estructura y verificar el balance de clases. Luego se dividieron los datos en entrenamiento y prueba, asegurando que ambas particiones mantuvieran la misma proporción de clases. También se aplicó escalamiento a los modelos que lo necesitaban (Regresión Logística y kNN), ya que estos dependen de la magnitud de las variables.

Después se entrenaron tres modelos:

- Regresión Logística  
- k-Nearest Neighbors (kNN)  
- Random Forest  

El entrenamiento consistió en permitir que cada modelo aprendiera patrones a partir de los datos de entrenamiento usando `.fit()`. Posteriormente se evaluaron en el conjunto de prueba utilizando métricas como Accuracy, Precision, Recall y F1-score.

Sin embargo, no nos quedamos solo con los valores por defecto. Se realizó una optimización de hiperparámetros usando GridSearchCV, probando diferentes configuraciones para encontrar la combinación que mejor desempeño ofreciera según el F1-score.

---

## Modelo con Mejor Desempeño

El modelo que obtuvo mejores resultados fue **kNN optimizado**, con:

- Accuracy: 0.9824  
- Precision: 0.9729  
- Recall: 1.0  
- F1-score: 0.9863  

Lo más importante es que logró **cero falsos negativos**.

En un problema médico como este, el error más grave es clasificar un tumor maligno como benigno. Ese tipo de error puede retrasar tratamiento y tener consecuencias muy serias. Por eso, más allá de la accuracy, el recall se vuelve una métrica clave.

El hecho de que el modelo tenga recall = 1.0 significa que detectó correctamente todos los casos positivos del conjunto de prueba.

---

## Sobre los Hiperparámetros

Durante la optimización se ajustaron distintos hiperparámetros:

- En Regresión Logística se ajustó el nivel de regularización.
- En kNN se probó con diferentes números de vecinos, tipos de ponderación y métricas de distancia.
- En Random Forest se ajustaron el número de árboles y su profundidad.

Los hiperparámetros influyen directamente en qué tan complejo o flexible es el modelo. Si el modelo es demasiado flexible puede sobreajustar; si es demasiado simple puede no capturar bien los patrones.

La optimización permitió encontrar un equilibrio adecuado, especialmente en el caso de kNN.

---

## Comparación General

| Modelo | Accuracy | Precision | Recall | F1-score |
|--------|----------|-----------|--------|----------|
| Logistic Regression | 0.9824 | 0.9861 | 0.9861 | 0.9861 |
| kNN | 0.9561 | 0.9589 | 0.9722 | 0.9655 |
| Random Forest | 0.9561 | 0.9589 | 0.9722 | 0.9655 |
| Logistic Regression (Opt) | 0.9737 | 0.9726 | 0.9861 | 0.9793 |
| kNN (Opt) | 0.9824 | 0.9729 | 1.0 | 0.9863 |
| Random Forest (Opt) | 0.9561 | 0.9589 | 0.9722 | 0.9655 |

Aunque varios modelos tuvieron resultados muy buenos, kNN optimizado fue el que mejor equilibró precisión y sensibilidad.

---

## Conclusión

Este laboratorio no fue solo sobre entrenar modelos, sino sobre entender cómo evaluarlos correctamente y cómo elegir el más adecuado según el contexto del problema.

Aprendimos que:

- Optimizar hiperparámetros puede marcar una gran diferencia.
- No siempre el modelo más complejo es el mejor.
- Las métricas deben elegirse según el impacto real del error.
- En aplicaciones médicas, minimizar falsos negativos es prioritario.

En general, este ejercicio permitió aplicar todo el flujo de un problema de Machine Learning supervisado, desde la exploración hasta el análisis crítico de resultados.
